# 画像認識

ここでは画像認識課題を，
WordNetプロジェクト（すべてのデータに名前をつけようとする第2次人工知能ブームのときから継続しているプロジェクト）が用意したMNISTデータセット（手書き数字のデータセット）を利用します．
MNISTデータセットとは縦横28ドットのモノクロ画像データであり，各画像には数字0から9まで10個のラベル（つまり，その画像が何の数字かということ）がついています．
28ｘ28=784なので，ベクトルとしては784次元です．

Pythonのライブラリとしてはscikit-learnを利用します．
まずはMNISTデータセットの準備を行います．

In [1]:
from sklearn import datasets, model_selection, svm, metrics

MNIST = datasets.load_digits()


MNISTデータセットの内容を確認しておきます．

In [ ]:
# どのくらいの大きさのデータセットか確認
print(MNIST.data.shape)

# 画像で表示するとどんなデータか確認
import matplotlib.pyplot as plt

# 以下のプログラムは数字画像10個を並べてうまく表示するため
# 「うまく表示」のために指定が多いなど複雑化しています．
# 気にしないでください
rows, cols = 2, 5 # 2行5列に並べたい
fg, ax = plt.subplots(rows, cols, figsize=(10.0, 4.0), tight_layout=True)
for i in range(rows * cols):
    ax[i // cols][i % cols].imshow(
        # // は整数除算，たとえば 3/2=1.5 だけど 3//2=1，% は整数剰余
        MNIST.images[i],
        cmap="binary",
        interpolation="nearest"
    )
    ax[i // cols][i % cols].set_title("target: %d"%(MNIST.target[i]))

plt.show()

以下でやりたいことは，
1. MNISTデータを2つ，学習用と検証用とに分ける
2. 学習用データで機械学習を実行する
3. 検証データで判定間違いがどれだけあるかなどを調べる

です．
学習の手法ですがサポートベクターマシンSVMと呼ばれる手法で実行しています．
機械学習の手法には他にもいろいろとあります．

問：スコアはいくつになっただろうか．報告してください．

In [ ]:
from sklearn.model_selection import train_test_split

data, labels = MNIST.data / 255, MNIST.target
data_size = len(labels)
# 変数名としてtrainで始まるのはトレーニング用，testで始まるのは検証用です
# 最初は両者を4：1に分けてみます
# train_test_split関数ではこれらをランダムに割り振るので
# 他の受講者とは多少違った結果になります
train_size, test_size = 8 * data_size // 10, 2 * data_size // 10
train_data, test_data, train_labels, test_labels = train_test_split(
    data,
    labels,
    test_size=test_size,
    train_size=train_size
)

# サポートベクトルマシンSVMという手法を使います
from sklearn.svm import SVC

# 準備
svm = SVC(kernel="rbf", C=1, gamma=0.5, random_state=0)
# 学習
svm.fit(train_data, train_labels)
# 検証，この変数ラベルLlabelがMNISTで用意した0から9までです
predicted_labels = svm.predict(test_data)
score = svm.score(test_data, test_labels)

# 最後に検証用データでの正答率を表示します
print("score: %.3f"%(score))

成功したものや失敗したものなど検証結果を描画します．
targetが教師データとしてのラベル，predictが学習にもとづく予測です．
プログラムの複雑さは，上でしたように，表示のための工夫です．

問：認識に成功した数字はどれで，失敗した数字はどれだろうか．認識に失敗（誤認識）した数字を一つ報告してください．

In [ ]:
import matplotlib.pyplot as plt

rows, cols = 2, 5
fg, ax = plt.subplots(rows, cols, figsize=(10.0, 4.0), tight_layout=True)
for i in range(rows * cols):
    ax[i // cols][i % cols].imshow(
        test_data[i].reshape([8, 8]),
        cmap="binary",
        interpolation="nearest"
    )
    ax[i // cols][i % cols].set_title("target: %d, predict: %d"%(test_labels[i], predicted_labels[i]))

plt.show()

どの数字が良くて，どの数字が悪かったのでしょう．
ヒートマップに示しました．
対角線上に濃い色が並ぶのは正しく認識できたことを示します．一方で対角線上以外にもやや濃い色のところが見られます．

問：対角線上以外の濃い色になっている数字を一つ報告してください．

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(test_labels, predicted_labels)

import matplotlib.pyplot as plt
import seaborn as sns
sns.heatmap(conf_mat, annot=True, cmap="Reds")
plt.show()